In [24]:
# ========================================================
# Notebook 09 - Modelo Supervisado para Clasificación ICD
# ========================================================

import pandas as pd
import numpy as np
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.utils.class_weight import compute_class_weight

import joblib

# -----------------------------
# 1. Cargar datos procesados
# -----------------------------
BASE_DIR = Path("..")  # Ajusta si tu notebook está en otra carpeta
DATA_PATH = BASE_DIR / "processed" / "suelos_icd_registro.csv"

df = pd.read_csv(DATA_PATH)
print("Dimensiones:", df.shape)
df.head()

C:\Users\diego\AppData\Local\Temp\ipykernel_28352\579573247.py:24: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DATA_PATH)


Dimensiones: (35111, 105)


,secuencial,fecha_de_an_lisis,departamento,municipio,cultivo,estado,tiempo_de_establecimiento,topografia,drenaje,riego,...,icd_comp_zinc_olsen,icd_total_zinc_olsen,icd_comp_boro_disponible,icd_total_boro_disponible,icd_comp_hierro_doble_acido,icd_total_hierro_doble_acido,icd_comp_manganeso_doble_acido,icd_total_manganeso_doble_acido,icd_comp_zinc_doble_acido,icd_total_zinc_doble_acido
0,1,2014-07-01,NARIÑO,SAN ANDRÉS DE TUMACO,No Indica,No indica,No indica,No indica,No indica,No indica,...,1.0,0.990773,1.0,0.991738,0.0,0.489899,0.0,0.489899,0.0,0.489899
1,164,2014-05-09,CUNDINAMARCA,UNE,Lulo,Establecido,De 0 a 1 año,Plano,Buen drenaje,Aspersión,...,1.0,0.991251,1.0,0.981230,0.0,0.489950,0.0,0.489950,0.0,0.489950
2,165,2014-05-09,CUNDINAMARCA,UNE,Lulo,Establecido,De 0 a 1 año,Ondulado,Buen drenaje,Aspersión,...,1.0,0.967378,1.0,0.940961,0.0,0.454320,0.0,0.454320,0.0,0.454320
3,166,2014-05-09,CUNDINAMARCA,FUNZA,Guisantes,Por establecer,No indica,Plano,Buen drenaje,Goteo,...,1.0,0.947616,1.0,0.976663,0.0,0.475861,0.0,0.475861,0.0,0.475861
4,167,2014-05-09,VALLE DEL CAUCA,LA UNIÓN,Guanábana,Establecido,Mas de 10 años,Plano,Buen drenaje,Manguera,...,1.0,0.947470,1.0,0.550316,0.0,0.417193,0.0,0.417193,0.0,0.417193


In [25]:
# ----------------------------------------------------
# 2. Convertir el ICD continuo en clases categóricas
# ----------------------------------------------------

def icd_categoria(valor):
    if valor >= 0.95:
        return "Excelente"
    elif valor >= 0.85:
        return "Bueno"
    elif valor >= 0.70:
        return "Regular"
    else:
        return "Deficiente"

df["ICD_cat"] = df[[c for c in df.columns if c.startswith("icd_total_")]].mean(axis=1).apply(icd_categoria)

df["ICD_cat"].value_counts()


ICD_cat
Regular       23165
Bueno         10678
Deficiente     1268
Name: count, dtype: int64

In [26]:
# -------------------------------------------------------
# 3. Seleccionar variables de suelo como predictores (X)
# -------------------------------------------------------

X_cols = [
    c for c in df.columns
    if not c.startswith("icd_")
    and df[c].dtype in [np.float64, np.int64]
]

X = df[X_cols]
y = df["ICD_cat"]


In [27]:
# -----------------------------------------
# 4. Train/Test split con estratificación
# -----------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

# Calcular pesos según balance de clases
weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(zip(np.unique(y_train), weights))
class_weights


{'Bueno': np.float64(1.0961122211122212),
 'Deficiente': np.float64(9.22993340343498),
 'Regular': np.float64(0.505218525766471)}

In [28]:
# -----------------------------------------
# Modelo: RandomForestClassifier
# -----------------------------------------

model = RandomForestClassifier(
    n_estimators=250,
    random_state=42,
    class_weight=class_weights
)

pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("model", model)
])

pipeline.fit(X_train, y_train)


,steps,"[('scaler', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,copy,True
,with_mean,True
,with_std,True
,n_estimators,250
,criterion,'gini'
,max_depth,None
,min_samples_split,2


In [29]:
# -----------------------------------------
# 5. Evaluación del modelo
# -----------------------------------------

y_pred = pipeline.predict(X_test)

print("\n📌 REPORT:\n", classification_report(y_test, y_pred))
print("\n📌 MATRIZ DE CONFUSIÓN:\n", confusion_matrix(y_test, y_pred))



📌 REPORT:
               precision    recall  f1-score   support

       Bueno       0.95      0.95      0.95      2670
  Deficiente       0.97      0.89      0.93       317
     Regular       0.97      0.97      0.97      5791

    accuracy                           0.96      8778
   macro avg       0.96      0.94      0.95      8778
weighted avg       0.96      0.96      0.96      8778


📌 MATRIZ DE CONFUSIÓN:
 [[2541    0  129]
 [   0  282   35]
 [ 143    8 5640]]


In [30]:
# -----------------------------------------
# 6. Guardar Modelo Entrenado
# -----------------------------------------

MODEL_PATH = BASE_DIR / "models" / "modelo_icd.pkl"
MODEL_PATH.parent.mkdir(exist_ok=True)

joblib.dump(pipeline, MODEL_PATH)
print(f"Modelo guardado en: {MODEL_PATH}")


Modelo guardado en: ..\models\modelo_icd.pkl


In [32]:
sorted(df["cultivo"].dropna().unique())

['Abarco',
 'Acacia',
 'Acelga',
 'Achiote',
 'Achira',
 'Agraz',
 'Aguacate',
 'Ahuyama',
 'Aji',
 'Ajo',
 'Ajonjoli',
 'Albahaca',
 'Alcachofa',
 'Alfalfa',
 'Algodón',
 'Anturio',
 'Anón',
 'Apio',
 'Apio de monte',
 'Aromáticas',
 'Arracacha',
 'Arroz',
 'Arveja',
 'Arándano',
 'Asai',
 'Aster',
 'Avena',
 'Avena Cayuse',
 'Badea',
 'Balú',
 'Bananito',
 'Banano',
 'Banco de proteina forrajero',
 'Batata',
 'Berenjena',
 'Bijao',
 'Bijas',
 'Boton de Oro',
 'Botón de Oro',
 'Brevo',
 'Brillantina',
 'Brócoli',
 'Cacao',
 'Cacay',
 'Caducifolios',
 'Café',
 'Calabacín',
 'Calabaza',
 'Calas',
 'Caléndula',
 'Camu camu',
 'Cannabis',
 'Caucho',
 'Caña',
 'Caña Azucarera',
 'Caña Panelera',
 'Cañamo',
 'Cebada',
 'Cebolla',
 'Cebolla de Bulbo',
 'Cebolla de Rama',
 'Cebollín',
 'Cedro',
 'Cedro Nogal',
 'Chirimoya',
 'Chisgua',
 'Chontaduro',
 'Chía',
 'Cilantro',
 'Ciruela',
 'Clavel',
 'Cocculus',
 'Coliflor',
 'Copoazu',
 'Cubios',
 'Curuba',
 'Cítricos',
 'Cúrcuma',
 'Durazno',
 '